In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
#reading business json file using spark
business_df = spark.read.json('s3://bigdaaassignmentbucket/yelp_academic_dataset_business.json')
#converting from json into a dataframe
business_df.createOrReplaceTempView("business")
business_frame_df = spark.sql("SELECT * FROM business")
business_frame_df.show(5)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1606327692812_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+-------+----------+------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|           city|               hours|is_open|  latitude|   longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+-------+----------+------------+--------------------+-----------+------------+-----+-----+
|     10913 Bailey Rd|[,,,,,,, True,, T...|f9NumwFMBDn751xgF...|Active Life, Gun/...|      Cornelius|[11:0-20:0, 10:0-...|      1|35.4627242| -80.8526119|The Range At Lake...|      28031|          36|  3.5|   NC|
|8880 E Via Linda,...|[,,,,,,,,,,, True...|Yzvjg0SayhoZgCljU...|Health & Medical,...|     Scottsdale|                null|      1|33.5694041|-111.89

In [2]:
#dropping all the unwanted columns
business_frame_df = business_frame_df.drop('hours','latitude','longitude','postal_code')
#choosing open only businesses
openbusiness_data = business_frame_df[business_frame_df['is_open']==1]
openbusiness_data.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+----------+-------+--------------------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|      city|is_open|                name|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+----------+-------+--------------------+------------+-----+-----+
|     10913 Bailey Rd|[,,,,,,, True,, T...|f9NumwFMBDn751xgF...|Active Life, Gun/...| Cornelius|      1|The Range At Lake...|          36|  3.5|   NC|
|8880 E Via Linda,...|[,,,,,,,,,,, True...|Yzvjg0SayhoZgCljU...|Health & Medical,...|Scottsdale|      1|   Carlos Santo, NMD|           4|  5.0|   AZ|
|3554 Rue Notre-Da...|                null|XNoUzKckATkOD1hP6...|Pets, Pet Service...|  Montreal|      1|             Felinus|           5|  5.0|   QC|
|  4827 E Downing Cir|[,,,,,,,,, True,,...|51M2Kk903DFYI6gnB...|Home Services, Pl...|      Mes

In [3]:
#choosing automotive business among all businesses
automotive_data = openbusiness_data.filter("categories like '%Automotive%'")
#Dropping is-open column as it is no longer needed
automotive_data = automotive_data.drop('is_open')
automotive_data.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+----------+--------------------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|      city|                name|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+----------+--------------------+------------+-----+-----+
|1720 W Elliot Rd,...|[,,,,,,,,, True,,...|cKyLV5oWZJ2NudWgq...|Auto Repair, Auto...|   Gilbert|Oasis Auto Center...|          38|  4.5|   AZ|
| 6910 E Southern Ave|[,,,,,,,,, True,,...|ScYkbYNkDgCneBrD9...|Auto Repair, Oil ...|      Mesa|Junction Tire & A...|          18|  5.0|   AZ|
|1110 Saw Mill Run...|[,,,,,,,,, True,,...|7Dv4_HAxsxvadEsT5...|Automotive, Auto ...|Pittsburgh|Dependable Brakes...|           5|  5.0|   PA|
|      2915 Cherry Rd|[,,,,,,,,, True,,...|edQoeeBFUTYGwnUSE...|Auto Customizatio...| Rock Hill|            Car Guyz|          10|  4.5|   SC|

In [4]:
#getting the total number of rows and columns for the automotive business
print((automotive_data.count(), len(automotive_data.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(13149, 9)

In [5]:
#reading the yelp reviews json file
reviews_df = spark.read.json('s3://bigdaaassignmentbucket/yelp_academic_dataset_review.json')
reviews_df.createOrReplaceTempView("reviews")
reviews_data = spark.sql("SELECT * FROM reviews")
#converting json file into reviews dataframe
reviews = reviews_data.withColumnRenamed('stars','review_stars')
reviews.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+-------------------+-----+--------------------+------------+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|review_stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+------------+--------------------+------+--------------------+
|-MhfebM0QIsKt87iD...|   0|2015-04-15 05:21:16|    0|xQY8N_XvtGbearJ5X...|         2.0|As someone who ha...|     5|OwjRMXRC0KyPrIlcj...|
|lbrU8StCq3yDfr-QM...|   0|2013-12-07 03:16:52|    1|UmFMZ8PyXZTY2Qcwz...|         1.0|I am actually hor...|     1|nIJD_7ZXHq-FX8byP...|
|HQl28KMwrEKHqhFrr...|   0|2015-12-05 03:18:11|    0|LG2ZaYiOgpr2DK_90...|         5.0|I love Deagan's. ...|     1|V34qejxNsCbcgD8C0...|
|5JxlZaqCnk1MnbgRi...|   0|2011-05-27 05:30:52|    0|i6g_oA9Yf9Y31qt0w...|         1.0|Dismal, lukewarm,...|     0|ofKDkJKXSKZXu5xJN...|
|IS4cv902ykd8wj1TR...|   0|2017-01-14 21:

In [6]:
#merging both business and reviews dataframe to get reviews of the automotive.
combined_data = automotive_data.join(reviews, on=['business_id'], how='inner')
combined_data.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+--------------------+--------------------+------+-----------------+------------+-----+-----+----+-------------------+-----+--------------------+------------+--------------------+------+--------------------+
|         business_id|       address|          attributes|          categories|  city|             name|review_count|stars|state|cool|               date|funny|           review_id|review_stars|                text|useful|             user_id|
+--------------------+--------------+--------------------+--------------------+------+-----------------+------------+-----+-----+----+-------------------+-----+--------------------+------------+--------------------+------+--------------------+
|2YUfXT7A-3rsDdWb_...|8369 W Bell Rd|[,,,,,,,,, True,,...|Car Wash, Automot...|Peoria|Jacksons Car Wash|         140|  2.5|   AZ|   0|2015-07-03 19:39:32|    0|feTHDWTHatdKxRI_0...|         2.0|I agree with what...|     0|zHXaFli3mO7CRPzLd...|
|2YUfXT7A-3rsDdWb_...|83

In [7]:
#dropping few unwanted columns
combined_data = combined_data.drop('address','attributes','state','city')
#writing into csv file
combined_data.coalesce(1).write.csv("s3://bigdaaassignmentbucket/Reviews.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…